# Audio - ASR (Automatic Speech Recognition)

Reconocimiento automático del habla (ASR), también conocido como texto a voz (STT), es la tarea de transcribir un audio dado a texto. Tiene muchas aplicaciones, pero nosotros nos vamos a enfocar en la tarea de extraer informacion y generar datasets con este tipo de modelos.

## Whisper

Whisper es un modelo de reconocimiento de voz de propósito general. Está entrenado en un gran conjunto de datos de audio diverso y también es un modelo multitarea que puede realizar reconocimiento de voz multilingüe, traducción de voz y identificación de idioma.

podemos trabajar con whisper directamente desde la libreria de [transformers](https://huggingface.co/docs/transformers/model_doc/whisper), o con [openai-whisper](https://github.com/openai/whisper). en este caso vamos a usar la libreria trasnformers.


In [ ]:
!pip install --upgrade pip
!pip install --upgrade transformers accelerate
!pip install  tiktoken pandas openai requests


## Audio de Ejemplo

[Simulacro de audiencia ante Juzgado Cívico de Tonalá, Jalisco](https://www.youtube.com/watch?v=soxHGSSMTIg)

La manera mas facil de obtener el audio de ejemplo es con yt-dlp y ffmpeg. pero en este caso esta bloqueado detras de colab.

```sh
!yt-dlp -x --audio-format mp3 -o "audio.mp3" https://www.youtube.com/watch?v=g8X6Qix-QUA

```

por ahora vamos a obtenerlo ya convertido en audio desde nuestros buckets.

In [ ]:
import requests

audio_url = "https://storage.googleapis.com/tallerdp_publico/audio01.mp3"

response = requests.get(audio_url)

with open("audio01.mp3", "wb") as f:
    f.write(response.content)

## Transcripcion

Lo segundo que tenemos que hacer es decidir que tamaño de whisper queremos usar, hay que tener en cuenta que las versiones mas pequeñas de whisper no son muy buenas haciendo transcripciones en español.

Hasta hace algunos meses la unica opcion viable para mi era Whisper Large, pero con la reciente introduccion de Whisper Turbo ya podemos elegir entre velocidad y fiabilidad de la transcripcion.

|  Size  | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
|:------:|:----------:|:------------------:|:------------------:|:-------------:|:--------------:|
|  tiny  |    39 M    |     `tiny.en`      |       `tiny`       |     ~1 GB     |      ~10x      |
|  base  |    74 M    |     `base.en`      |       `base`       |     ~1 GB     |      ~7x       |
| small  |   244 M    |     `small.en`     |      `small`       |     ~2 GB     |      ~4x       |
| medium |   769 M    |    `medium.en`     |      `medium`      |     ~5 GB     |      ~2x       |
| large  |   1550 M   |        N/A         |      `large`       |    ~10 GB     |       1x       |
| turbo  |   809 M    |        N/A         |      `turbo`       |     ~6 GB     |      ~8x       |


Para nuestros ejemplos vamos a usar whisper turbo

In [ ]:
import os
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

# tenemos tarjeta de video? esto es muy importante cuando trabajos con whisper
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

#podemos usar un modelo mas pequeño, si no tenemos problemas con el idioma
model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

#job de audio->texto
whisper_pipeline = pipeline(
    "automatic-speech-recognition", #whisper tiene mas de 1 tarea
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    chunk_length_s=30, # bajar numero para reducir uso de memoria
    batch_size=16 #
)

#procesamos el mp3 y sacamos los timestamps, forzamos la deteccion a español por ahora
result = whisper_pipeline("audio01.mp3", generate_kwargs={"language": "spanish"},
                          return_timestamps=True)


#imprimimos el resultado, aqui deberiamos de almacenarlo en algun lugar.
print(result)

## Contexto

Con que tanta informacion estamos trabajando?  y de que manera podemos trabajr con ella?

In [ ]:
import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")
tokens = encoding.encode(result['text'])
print(len(tokens))

Es un numero de tokens razonable para poder hacer extraccion de entidades usando alguna variante de [BERT](https://huggingface.co/mrm8488/bert-spanish-cased-finetuned-ner). Pero para no tener algun problema podemos trabajar con los chunks, o pedazos de transcripcion que creo whisper.

In [ ]:
import pandas as pd
#usamos Bert para sacar entidades usando NER
ner_pipeline = pipeline(task="ner",
                        model="mrm8488/bert-spanish-cased-finetuned-ner",
                        device=device,
                        aggregation_strategy="simple")


all_entities = []
for chunk in result["chunks"]:
    ner = ner_pipeline(chunk["text"])
    entities = [
                {
                    "type": pred["entity_group"],
                    "score": round(pred["score"], 4),
                    "word": pred["word"],
                    "start": pred["start"],
                    "end": pred["end"],
                }
                for pred in ner
            ]
    for entity in entities:
      all_entities.append(entity)

df = pd.DataFrame(all_entities)
df

In [ ]:
# todas las personas y ubicaciones unidas
df_filtered = df[df["type"].isin(["PER", "LOC"])]
df_unique = df_filtered.drop_duplicates(subset=["type", "word"])

df_unique

## Extraccion usando LLMs


Tambien podemos generar extracciones usando modelos de lenguaje, aqui ya podemos extraer informacion mas compleja.



In [ ]:
from openai import OpenAI
from google.colab import userdata
client = OpenAI(
    api_key=userdata.get('GOOGLE_API_KEY'),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

prompt = f"""
De la siguiente transcripcion de un audio:

```
{result['text']}
```

Extrae los siguientes datos:
- Nombre del juez (obligatorio)
-  Lista de personas en la audiencia (obligatorio), donde cada persona incluye:
 - Nombre (opcional)
 - Cargo (opcional)
- Resumen de toda la audiencia (obligatorio)
- Tipo de audiencia (opcional)
- Ubicación de la audiencia (opcional)
- Sentencia de la audiencia (opcional)
"""

response = client.chat.completions.create(
  model="gemini-2.5-flash",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "Eres un sistema de extracción de información de transcripciones de audio, solo generas JSON valido de acuerdo con el schema proporcionado."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": prompt
        }
      ]
    }
  ],
  response_format={
    "type": "json_schema",
    "json_schema": {
      "name": "audience",
      "strict": True,
      "schema": {
        "type": "object",
        "properties": {
          "judge_name": {
            "type": "string",
            "description": "Nombre del juez que preside la audiencia."
          },
          "participants": {
            "type": "array",
            "description": "Lista de personas en la audiencia.",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string",
                  "description": "Nombre de la persona."
                },
                "position": {
                  "type": "string",
                  "description": "Cargo de la persona en la audiencia."
                }
              },
              "required": [
                "name",
                "position"
              ],
              "additionalProperties": False
            }
          },
          "audience_summary": {
            "type": "string",
            "description": "Resumen de toda la audiencia."
          },
          "audience_type": {
            "type": "string",
            "description": "Tipo de audiencia."
          },
          "audience_location": {
            "type": "string",
            "description": "Ubicación donde se lleva a cabo la audiencia."
          },
          "audience_sentence": {
            "type": "string",
            "description": "Sentencia resultante de la audiencia."
          }
        },
        "required": [
          "judge_name",
          "participants",
          "audience_summary",
          "audience_type",
          "audience_location",
          "audience_sentence"
        ],
        "additionalProperties": False
      }
    }
  },
  temperature=0.1
)

print(response.choices[0].message)

In [ ]:
import json
data = json.loads(response.choices[0].message.content)

print(json.dumps(data, indent=4))


## Gemini

Antes de Gemini 2.0 flash esta era la manera indeal de extraer informacion de audio, pero ahora todo este proceso es mucho mas rapido y barato usando directamente Gemini sin pasos intermedios.

Hay casos en los que todavia se recomienda usar las opciones anteriores.

In [ ]:
import os
from google import genai
from google.genai import types
from google.colab import userdata



def generate():
    client = genai.Client(
        api_key=userdata.get("GOOGLE_API_KEY"),
    )

    model = "gemini-2.5-flash"
    audio_file = client.files.upload(file="audio01.mp3")
    prompt = """Del audio adjuntado  extrae los siguientes datos:

- Nombre del juez (obligatorio)
-  Lista de personas en la audiencia (obligatorio), donde cada persona incluye:
 - Nombre (opcional)
 - Cargo (opcional)
- Estado de animo / Sentimiento (opcional)
- Resumen de toda la audiencia (obligatorio)
- Tipo de audiencia (opcional)
- Calidad del audio (opcional)
- Ubicación de la audiencia (opcional)
- Sentencia de la audiencia (opcional)"""

    contents=[prompt, audio_file]

    generate_content_config = types.GenerateContentConfig(
        thinking_config = types.ThinkingConfig(
            thinking_budget=-1,
        ),
        response_mime_type="application/json",
        response_schema=genai.types.Schema(
            type = genai.types.Type.OBJECT,
            properties = {
                "nombre_juez": genai.types.Schema(
                    type = genai.types.Type.STRING,
                ),
                "participantes": genai.types.Schema(
                    type = genai.types.Type.ARRAY,
                    items = genai.types.Schema(
                        type = genai.types.Type.OBJECT,
                        properties = {
                            "nombre": genai.types.Schema(
                                type = genai.types.Type.STRING,
                            ),
                            "cargp": genai.types.Schema(
                                type = genai.types.Type.STRING,
                            ),
                            "sentimiento": genai.types.Schema(
                                type = genai.types.Type.STRING,
                            ),
                        },
                    ),
                ),
                "resumen": genai.types.Schema(
                    type = genai.types.Type.STRING,
                ),
                "tipo_audiencia": genai.types.Schema(
                    type = genai.types.Type.STRING,
                ),
                "calidad_audio": genai.types.Schema(
                    type = genai.types.Type.STRING,
                ),
                "ubicacio_audiencia": genai.types.Schema(
                    type = genai.types.Type.STRING,
                ),
            },
        ),
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end="")

generate()

# Ejercicio

- Que tantas veces se dijo por favor?
- Cuantas muletillas vocales se usaron en la audiencia?  